## AlteryxFunctionsTranspiler

### Motivação
Ao considerar a criação de um transpiler de expressões do Alteryx, minhas motivações estão diretamente relacionadas à necessidade de adaptar ou reaproveitar essas expressões em outros sistemas ou ferramentas com sintaxes diferentes.

Sabemos que o Alteryx possui sua própria linguagem de expressões, amplamente usada em operações de transformação e processamento de dados. A documentação oficial sobre funções e expressões do Alteryx <https://help.alteryx.com/20232/en/designer/functions.html> ajuda a entender a lógica por trás dessas funções.

Converter expressões do Alteryx para Python, especialmente quando o objetivo é vincular essas expressões a funções UDF (User Defined Functions) em PySpark, apresenta uma série de desafios técnicos e conceituais. Vou listar os principais desafios que surgem nesse processo:

#### 1. Diferenças de Sintaxe e Semântica

As expressões do Alteryx têm uma sintaxe e comportamento próprios, o que pode gerar dificuldades na conversão direta para Python ou PySpark. Muitas das funções e operadores do Alteryx possuem um comportamento que pode ser diferente das bibliotecas padrão de Python ou PySpark. Exemplos incluem:

- Funções matemáticas e de manipulação de texto: O Alteryx possui funções nativas específicas (como Trim(), Length(), DateTimeDiff(), etc.) que precisam ser mapeadas para funções equivalentes no Python ou PySpark.

- Funções de agregação: Algumas funções no Alteryx (ex. Sum(), Average(), Min(), etc.) podem precisar de um tratamento especial ao serem convertidas para o contexto distribuído do PySpark.
A conversão semântica não se limita à simples substituição de funções, mas também à adaptação do comportamento de execução, como otimização de cálculos e o manuseio de tipos de dados.


#### 2. Diferente Modelagem de Dados
- Alteryx trabalha com um modelo de dados mais centrado em tabelas e campos, enquanto o PySpark lida com RDDs ou DataFrames distribuídos. Isso implica que a maneira como dados são manipulados e processados em Alteryx pode ser diferente da abordagem no PySpark.

- UDFs no PySpark são funções definidas pelo usuário que podem ser aplicadas a cada linha de um DataFrame, e sua implementação precisa ser eficiente em termos de paralelismo. Converter expressões do Alteryx para UDFs pode envolver transformações significativas no modelo de dados, que precisam ser ajustadas para funcionar de forma distribuída.

### Características das expressões do Alteryx

- As expressões no Alteryx são altamente estruturadas e possuem uma gramática relativamente bem definida.
- Existe um conjunto limitado de funções, operadores e sintaxe específicas da ferramenta.
- A validação das expressões é importante para garantir que o código gerado funcione corretamente no ambiente de destino.

#### Dado esse contexto, a abordagem ideal depende de dois fatores principais:

- Complexidade e variação nas expressões: As expressões são sempre bem definidas ou possuem variações ambíguas?
- Exigência de precisão: Pequenos erros na conversão podem comprometer o resultado final?

### Usando ANTLR

#### Por que ANTLR seria ideal?

- Gramática bem definida: Se as expressões do Alteryx seguem uma estrutura previsível, ANTLR é perfeito para processar essas expressões com precisão.
- Validação rigorosa: ANTLR valida automaticamente contra a gramática, garantindo que expressões malformadas sejam rejeitadas ou corrigidas.
- Modularidade: Você pode criar uma gramática para Alteryx e outra para o formato da ferramenta Formula, além de ajustar mapeamentos para conversão.
- Performance: Após a implementação, a conversão é rápida e altamente otimizada.

#### Desafios:

- Necessidade de construir uma gramática para o Alteryx e o formato Formula.
- Esforço inicial maior para implementação.

### Usando LLM

#### Por que LLM seria útil?

- Flexibilidade: Se as expressões do Alteryx não são perfeitamente padronizadas (por exemplo, combinações complexas ou inputs ambíguos), uma LLM pode interpretar melhor os padrões e realizar a conversão.
- Prototipagem rápida: É possível iniciar sem criar uma gramática formal, reduzindo o tempo de desenvolvimento inicial.
- Generalização: Além de converter expressões, a LLM pode lidar com comentários, anotações ou outros metadados presentes.

#### Desafios:

- Imprecisão: As saídas podem ser inconsistentes, especialmente em expressões complexas ou raras.
- Falta de controle: Dificuldades em garantir validação formal das expressões geradas.
- Dependência de treinamento: Uma LLM pode não ter conhecimento prévio das expressões do Alteryx ou da ferramenta Formula, exigindo ajustes personalizados.

#### Comparação Direta

| Critério             | ANTLR                                | LLM                                      |
|----------------------|--------------------------------------|------------------------------------------|
| Precisão             | Alta (dependendo da gramática)       | Média (dependendo do modelo e ajuste)    |
| Escalabilidade       | Boa                                   | Limitada por custo computacional         |
| Facilidade de uso    | Média (exige aprendizado inicial)    | Alta (sem necessidade de gramática)      |
| Velocidade de execução | Alta (após implementação)           | Média (requer inferências)               |
| Flexibilidade        | Baixa                                | Alta                                     |

### Cenário Escolhido: Híbrido

- ANTLR para o core: Processa a maior parte das expressões do Alteryx com uma gramática bem definida.
- LLM para refinamento: Aplica heurísticas ou resolve ambiguidades não previstas pela gramática, complementando o processo.

Essa abordagem garante precisão e flexibilidade, otimizando esforços de desenvolvimento e minimizando erros.

### POC

In [8]:
from antlr4 import *
from AlteryxFunctionsTranspiler.AlteryxFunctionsLexer import AlteryxFunctionsLexer
from AlteryxFunctionsTranspiler.AlteryxFunctionsParser import AlteryxFunctionsParser
from AlteryxFunctionsTranspiler.AlteryxFunctionsVisitor import AlteryxFunctionsVisitor

In [9]:
from AlteryxFunctionsTranspiler.AlteryxToPython import AlteryxToPythonVisitor

In [10]:
def transpile_alteryx_to_python(alteryx_expression):
    input_stream = InputStream(alteryx_expression)
    lexer = AlteryxFunctionsLexer(input_stream)
    stream = CommonTokenStream(lexer)
    parser = AlteryxFunctionsParser(stream)
    tree = parser.prog()

    visitor = AlteryxToPythonVisitor()
    return visitor.visit(tree)

### ALTERYX EXPRESSION: TONUMBER

In [11]:
alteryx_expression = 'ToNumber("42")'
python_code = transpile_alteryx_to_python(alteryx_expression)
print("Python Code:", python_code)
print("Execution Result:", eval(python_code))

Python Code: float("42")
Execution Result: 42.0


### ALTERYX EXPRESSION: IF TOBOOLEAN("1") THEN TOSTRING(100) ELSE "False" ENDIF

In [12]:
alteryx_expression = 'IF TOBOOLEAN("1") THEN TOSTRING(100) ELSE TONUMBER("200") ENDIF'
python_code = transpile_alteryx_to_python(alteryx_expression)
print("Python Code:", python_code)
print("Execution Result:", eval(python_code))

Python Code: (str(100) if None else float("200"))
Execution Result: 200.0


### ALTERYX EXPRESSION COMPLEX:

In [13]:
alteryx_expression = """
IF TOBOOLEAN("1")

    THEN
        TOSTRING(100)

    ELSE
        IF TOBOOLEAN("1")
            THEN ToNumber("42")

        ELSE ToNumber("49")

    ENDIF

ENDIF

"""


In [14]:
python_code = transpile_alteryx_to_python(alteryx_expression)
print("Python Code:", python_code)
print("Execution Result:", eval(python_code))

Python Code: (str(100) if None else (float("42") if None else float("49")))
Execution Result: 49.0


#### Gerando o Código Python (UDF) a partir da AST

Agora, precisamos percorrer a árvore de sintaxe (AST) gerada pelo ANTLR e gerar o código Python equivalente, que define uma UDF para a expressão em PySpark.

#### Exemplo não testado ainda!

```python
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def generate_udf_code(ast):
    condition = ast['condition']
    true_value = ast['true_value']
    false_value = ast['false_value']
    
    code = f"""
def udf_function({condition}):
    if {condition}:
        return {true_value}
    else:
        return {false_value}
    """
    return code

ast = {
    'type': 'IF',
    'condition': '[Age] > 30',
    'true_value': '"Rafael"',
    'false_value': '"Jose"'
}

udf_code = generate_udf_code(ast)

```



### Conclusão

A conversão de expressões do Alteryx para funções UDF em PySpark é um processo complexo, que envolve a adaptação de sintaxe, tipos de dados, e comportamentos de execução. Embora o Alteryx tenha uma abordagem mais simples e centrada em dados tabulares, o PySpark exige uma adaptação para trabalhar com dados distribuídos, o que pode afetar o desempenho e a precisão das conversões. Uma abordagem cuidadosa de mapeamento de funções, validação e teste é essencial para garantir que a conversão preserve a integridade e a eficiência das expressões originais do Alteryx.